# 用户账号处理



In [4]:
%load_ext dotenv
%dotenv

import sqlite3
import pandas as pd
import os
import json
import hashlib
import pgOperation
from sqlalchemy.types import VARCHAR, Float, Integer, Date, Numeric
pd.options.mode.chained_assignment = None

ana = sqlite3.connect('./game_analysis.db')
pg = pgOperation.PgOperation('localhost', '5432', os.getenv('PG_USER'), '1', 'tfm')

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
user = pg.readTable('users');
user.drop_duplicates(subset=['id'],keep='last', inplace=True)
user['user_name'] = user['name']
user['is_id'] = 1
user = user.loc[:,['name', 'user_name', 'is_id']]
user_csv = pd.read_csv('data/alias_to_user_name.csv')
user_csv['is_id'] = 0
# user_name to lowercase
user['name'] = user['name'].str.lower()
user['user_name'] = user['user_name'].str.lower()
user_csv.columns = user.columns
user_df = pd.concat([user,user_csv],axis=0).drop_duplicates(keep='first', inplace=False)
pg.writeDfToPg(user_df, 'ods_user_alias') # 用于匹配用户id

/home/cyan/Documents/gh/terraforming-mars-data/pgOperation.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,con=conn)


In [6]:
# 在存在用户别名的情况下，还有用户名匹配哪些是缺漏的
sql = """
select c.player, '' as 实际用户名, count(*) as 出现次数
from (select a.*, b.user_name
      from ods_game_2023_results as a
               left join ods_user_alias as b on a.player = b.name) as c
where c.user_name is null
group by c.player
order by count(*) desc;
"""
pg.readSql(sql).to_csv('data/missing_user_name.csv', index=False)

/home/cyan/Documents/gh/terraforming-mars-data/pgOperation.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,con=conn)


## Supabase

In [8]:

# import os
# from supabase import create_client, Client

# url: str = os.environ.get("SUPABASE_URL")
# key: str = os.environ.get("SUPABASE_KEY")
# supabase: Client = create_client(url, key)

In [9]:
# 生成用户账号密码表，并上传到supabase用于鉴权
supabase_user_df = pd.read_sql_query("select * from users_2022", cnx_o);
# supabase_user_df['is_superuser'] = 0
# supabase_user_df['is_active'] = 1
# supabase_user_df['is_staff'] = 0

In [12]:
supabase_user_df.to_csv('user.csv', index=False)

In [ ]:
pd.read_sql_query("SELECT * FROM users", cnx_o)

,id,name,password
0,3a2ae147d214,dawn,a3b0417b947057c57b4e7dbabac810fc
1,f809d54498c,友人,62cd53b114f130f42067ea4034208354
2,f1047bf31e48,冰雪,9656d15550c3807ced8f09d14f0af618
3,55ef2445c90c,乙木,96e79218965eb72c92a549dd5a330112
4,1b736baf443a,NoswordKnight,241ce91f699af0dc12f9fbf99b31f6b6
...,...,...,...
5505,46b2753edcdf,xu,8d91d563f78e48d6fb381bae1baf28ce
5506,a7c93d48d96d,biaodi,9b19d88409968a39643d8cf6183192bc
5507,cb42a581e99b,阿斯刚超,83fd1c87b4d2d8a4e55a56d81614823d
5508,cb42a581e99b,阿斯刚超,83fd1c87b4d2d8a4e55a56d81614823d


In [ ]:
pd.read_sql_query("SELECT * FROM ods_user_alias", ana_o)

,name,user_name,is_id
0,文生,文生,1
1,星雨日和,星雨日和,1
2,outwhite,outwhite,1
3,冰雪,冰雪,1
4,庄生,庄生,1
...,...,...,...
5095,智慧球,宵暗,0
5096,魔术本就逆天而行,宵暗,0
5097,园,yuan,0
5098,sssssss,silenced,0
